In [ ]:
import numpy as np
import scipy.io
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate, KFold, GridSearchCV
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import functions as F

import xgboost as xgb

import matplotlib.pyplot as plt


Loading mat File

In [ ]:
# mat_file_path = './results/results_CSSP_tau5.mat'
mat_file_path = './results/results_CSSP_tau15.mat'

data = scipy.io.loadmat(mat_file_path)
cell_array = data['results']
mat_file_path = './results/labels.mat'

label = scipy.io.loadmat(mat_file_path)
label = label["Label"].reshape(-1)
print(f"Cell array input structure: {cell_array.shape} which has 3 sub structure and 10 models.")

In [ ]:
for mdl in range(cell_array.shape[1]):
    print(f"mdl variable number is:{mdl}")
    dataset = cell_array[1,mdl]
    print(f"Dataset shape is: {dataset.shape} and label is: {label.shape} and type of the var is: {type(label)}")



In [ ]:
models = {
    "SVM_linear" : svm.SVC(kernel="linear"),
    "SVM_rbf" : svm.SVC(kernel="rbf"),
    "LogisticReg" : LogisticRegression(),
    "XGboost" : xgb.XGBClassifier(),
    "LDA": LinearDiscriminantAnalysis()
}

print(type(models))
print(models.keys())

k = 10
kf = KFold(n_splits=k)

scoring = {'accuracy':metrics.make_scorer(metrics.accuracy_score),
           'precision':metrics.make_scorer(metrics.precision_score),
           'recall':metrics.make_scorer(metrics.recall_score),
           'f1_score':metrics.make_scorer(metrics.f1_score)}


results_performance = {}
# results_accuracy = {}

results_accuracy = {name: [] for name in models.keys()}

for mdl in range(cell_array.shape[1]):
    dataset = cell_array[0,mdl]
    print(f"Dataset shape is: {dataset.shape} and label is: {label.shape} and type of the var is: {type(label)}")
    for name, model in models.items():
        scores = cross_validate(model, dataset, label, cv=kf, scoring=scoring, n_jobs=-1)

        result_name = f"model_{mdl+1}_{name}"

        print(result_name)
        results_performance[result_name] = scores
        results_accuracy[name].append(np.mean(scores['test_accuracy']))
        


In [ ]:
# print(scores.keys())
# print(results_performance.keys())

print(results_accuracy.keys())
a = np.array(results_accuracy['SVM_linear'])
print(type(a))
print(a.shape)

In [ ]:
# Generate some data
x = np.linspace(2, 20, num=10)
# x = np.arange(2,22,2)
print(x)
# Plot the data
plt.figure(figsize=(8, 6))  # Set the figure size

for model,accuracies in results_accuracy.items():
    # print(type(accuracies))
    # print(len(accuracies))
    accuracies = np.array(accuracies)*100
    plt.plot(x,accuracies,label=model, marker='o', linestyle='-')

# Add labels and title
plt.xlabel('Number of Features')  # x-axis label
plt.ylabel('Accuracy (%)')  # y-axis label
plt.title('Accuracy for features')  # Plot title

# Add legend
plt.legend(loc='upper left')  # Display legend in upper right corner

# # Customize ticks
plt.xticks(np.arange(2, 22, 2))  # Set x-axis ticks from 0 to 10 in steps of 1
# plt.yticks(np.arange(-1, 1.5, 0.5))  # Set y-axis ticks from -1 to 1.5 in steps of 0.5

# Add grid
plt.grid(True)

# Show plot
plt.show()